In [1]:
import strawberryfields as sf
from strawberryfields import ops
import matplotlib.pyplot as plt
from scipy.stats import poisson
import pennylane as qml
from pennylane import numpy as np

np.random.seed(0)

import numpy as np
import random
import math

from copy import deepcopy

In [2]:
#Select a value for the phase to be estimmated.
theta = np.pi/2

In [3]:
#Simulating the physical experiment.
prog = sf.Program(2)
phi = prog.params('phi')
alphap = prog.params('alpha')
thetaa = prog.params('theta')
with prog.context as q:
    ops.Dgate(alphap) | q[0]
    ops.Fock(0) | q[1]
    ops.BSgate() | (q[0],q[1])
    ops.Rgate(thetaa) | q[0]
    ops.Rgate(phi) | q[1]
    ops.BSgate() | (q[0],q[1])
    ops.MeasureFock() | [0, 1]
    

eng = sf.Engine('fock', backend_options={'cutoff_dim': 5})
cond = True
result = None
while cond:
    result = eng.run(prog, args={'phi': np.pi/4,'alpha': np.sqrt(10),'theta':theta})
    if(result.samples[0][0]+result.samples[0][1]!=0):
        break
print("Output Fock states:", result.samples)

Output Fock states: [[0 4]]


In [4]:
#Defining the Cost Function
ss = 10
def cost_function(phi,alpha):
    cost_sum = 0
    for i in range(ss):
        cond = True
        output = None
        while cond:
            output = eng.run(prog, args={'phi': phi,'alpha': alpha,'theta':np.pi/2})
            if(output.samples[0][0]+output.samples[0][1]!=0):
                break
        ket_i = output.samples[0]/np.sqrt(((output.samples[0][0])**2)+((output.samples[0][1])**2))
        target_state = [0,1]
    
        cost_sum = cost_sum + np.abs(np.dot(ket_i,target_state) - 1)
    cost = cost_sum/ss
    return cost

In [5]:
#Initialise the Q Table
d =100
Q_table_global = np.zeros([d,3])

In [6]:
#Defining the hyperparameters
n_episodes = 200
max_iter_episode = 50
exploration_proba = 0.1
exploration_decreasing_decay = 0.01
min_exploration_proba = 0.01
gamma = 0.4
lr = 0.1
action_space = [-1, 0, 1]

In [7]:
#defining a function to calculate variance:
def var_ftn(lst):
    s = 0+0j
    vh = 0
    p_list_pre = np.zeros(d)
    p_list = np.zeros(d)
    for i in range(len(lst)):
        p_list_pre[lst[i]] = p_list_pre[lst[i]]+1
    p_list = p_list_pre/len(lst)
    for j in range(len(p_list)):
        s = s+p_list[j]*np.exp(1j*(((j)*2*np.pi/d)-theta))
    abs_s = np.abs(s)
    vh = (abs_s)**(-2)-1
    return vh

In [8]:
#Choose mean number of photons in the poisson distribution
N=4

In [9]:
#Implementing the Q Learning Algorithm
rewards_per_episode = list()
q_table_consolidated = list()
for l in range(N,N+1):
    alph = np.sqrt(l)
    for e in range(n_episodes):
        current_state = random.randint(0, 99)
        total_episode_reward = 0
        prev_cost = 0
        for i in range(max_iter_episode): 
            if np.random.uniform(0,1) < exploration_proba:
                action = action_space[random.choice([0,1,2])]
            else:
                action = action_space[np.argmax(Q_table_global[current_state][:])]
            next_state = current_state+action
            if(next_state <= -1):
                next_state = d-1
            if(next_state >= d):
                next_state = 0
            reward = 1000*(1 - cost_function(next_state*2*np.pi/d,alph))
            Q_table_global[current_state][action_space.index(action)] = (1-lr) * Q_table_global[current_state][action_space.index(action)] +lr*(reward + gamma*max(Q_table_global[next_state][:]))
            total_episode_reward = total_episode_reward + reward
            current_state = next_state
        print("Episode done, Episode number: ",e)
        exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay*e))
        rewards_per_episode.append(total_episode_reward)
        q_table_consolidated.append(deepcopy(Q_table_global))

Episode done, Episode number:  0
Episode done, Episode number:  1
Episode done, Episode number:  2
Episode done, Episode number:  3
Episode done, Episode number:  4
Episode done, Episode number:  5
Episode done, Episode number:  6
Episode done, Episode number:  7
Episode done, Episode number:  8
Episode done, Episode number:  9
Episode done, Episode number:  10
Episode done, Episode number:  11
Episode done, Episode number:  12
Episode done, Episode number:  13
Episode done, Episode number:  14
Episode done, Episode number:  15
Episode done, Episode number:  16
Episode done, Episode number:  17
Episode done, Episode number:  18
Episode done, Episode number:  19
Episode done, Episode number:  20
Episode done, Episode number:  21
Episode done, Episode number:  22
Episode done, Episode number:  23
Episode done, Episode number:  24
Episode done, Episode number:  25
Episode done, Episode number:  26
Episode done, Episode number:  27
Episode done, Episode number:  28
Episode done, Episode nu

In [10]:
#Calculating the Variance
act_0 = np.zeros([n_episodes,d])
phase_estimate_list_index = list()
for i in range(len(q_table_consolidated)):
    for j in range(len(q_table_consolidated[i])):
        act_0[i][j] = q_table_consolidated[i][j][1]
for i in range(len(act_0)):
    phase_estimate_list_index.append(np.argmax(act_0[i]))
import math
count = 0
s_list = list()
s_abs_list = list()
vh_list = list()
p_list_pre = np.zeros(100)
p_list = np.zeros(100)
for i in range(len(phase_estimate_list_index)):
    count = count+1
    p_list_pre[phase_estimate_list_index[i]] = p_list_pre[phase_estimate_list_index[i]]+1
    p_list = p_list_pre/count
    s = 0+0j
    for j in range(len(p_list)):
        s = s+p_list[j]*np.exp(1j*(((j)*2*np.pi/d)-theta))
    abs_s = np.abs(s)
    vh = (abs_s)**(-2)-1
    s_list.append(s)
    s_abs_list.append(abs_s)
    vh_list.append(vh)
diff_list = list()
for i in range(len(vh_list)):
    diff_list.append(vh_list[i]-(1/np.sqrt(4)))
    if(diff_list[i] < 0):
        diff_list[i] = 10000
print(vh_list[np.argmin(diff_list)])

0.5014966221522683


In [11]:
print(1/np.sqrt(4))

0.5


In [14]:
print(Q_table_global)

[[6.52586294e+02 9.98525254e+02 6.46481253e+02]
 [7.45942934e+02 2.86858106e+02 4.10541267e+02]
 [2.67071781e+02 2.35015166e+02 4.80610203e+02]
 [2.41666040e+02 6.44550351e+01 4.31040737e+02]
 [6.81698458e+01 7.13981384e+01 5.98591173e+02]
 [1.61334037e+02 3.29959139e+02 1.06228481e+03]
 [6.93925338e+02 8.61592360e+02 1.33877830e+03]
 [1.28963669e+03 1.33057712e+03 1.37764544e+03]
 [1.36169718e+03 1.05702700e+03 7.67364296e+02]
 [1.17293583e+03 1.36465161e+03 9.82811045e+02]
 [1.30809568e+03 1.44459927e+03 1.29012093e+03]
 [1.41566522e+03 6.15216373e+02 5.29856388e+02]
 [1.18473095e+03 3.31156190e+02 2.76994669e+02]
 [9.39957928e+02 9.28481555e+01 3.92212758e+02]
 [8.85908779e+02 5.39357829e+02 6.13198495e+02]
 [9.32013453e+02 8.29189595e+02 1.57704109e+03]
 [1.55638153e+03 1.59938736e+03 1.61363282e+03]
 [1.62689286e+03 1.52136406e+03 1.51876420e+03]
 [1.58910547e+03 5.91063704e+02 1.13535816e+03]
 [1.39736983e+03 1.63369465e+03 1.31889464e+03]
 [1.53247029e+03 7.29413200e+02 1.060388